In [1]:
import numpy as np


import string
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, BooleanType
from pyspark.ml import Pipeline, PipelineModel

from nlp_cl_start import data_tokenizer

In [2]:
import pandas as pd

In [3]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder
        .master("local[4]")
        .appName("yelp_academic")
        .getOrCreate()
        )
sc = spark.sparkContext

In [17]:
#x = PipelineModel.read()

In [22]:
spark

In [28]:
#p = x.load('tfidfkmeans/')


In [7]:
#pipe = PipelineModel.load('tfidf/')
pipe = PipelineModel.load('pipe-idf-5-7-18/')

In [6]:
us_ex = pd.read_csv('user_from_scr_clidf18.csv')

In [12]:
np.unique(us_ex.user_cl, return_counts=True)

(array([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 12, 13, 16, 17]),
 array([71, 81, 30, 47, 38, 20, 17, 22,  8, 20,  4, 10,  9,  8]))

In [8]:
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
rv_s = db['reviews_scrap']

In [9]:
rv_s.find()

{'_id': ObjectId('5bfb5ed579884720729dd209'),
 'id': 'ErsYo_9v8--feFCy92tymQ',
 'rating': '5.0',
 'text': 'Fantastic Thai food with warm service. Authentic flavors and price is reasonable. Everything is good here as we tried most of the menu.',
 'category': '\nThai\n',
 'date': '10/13/2018',
 'alias': 'srivilai-thai-cuisine-seattle',
 'user_id': '_3KTfz0hcPl_yq4jOyOPiA'}